## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode
from multiprocess import Pool
from math import ceil
from dotenv import dotenv_values
from itertools import islice
from IPython.core.display import display, HTML

from contract_utils import execute_bash, instantiate_contract, execute_contract, query_contract, get_ipfs_cid_from_str, get_proofs, ParseOutput, execute_contract_sdk, send_coins


ipfs_client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 1000
NUMBER_OF_ACTIVATED_PARTICIPANTS = 10
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [get_ipfs_cid_from_str(_nickname + '_avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_000_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(13)
COEF_DOWN = str(7)
TARGET_CLAIM = str(2)

WALLET_ADDRESS = dotenv_values('.env')['WALLET_ADDRESS']
WALLET_SEED = dotenv_values('.env')['WALLET_SEED']
DISPLAY_TX_EXECUTION = False
TEST_ACCOUNT_ADDRESS = 'bostrom1mxdtr8lruutugqtxgpw2sf2tl2mhzlq5fd2du0'
TEST_ACCOUNT_SEED = 'end spread blind steak install glare pride pony switch exit zone faint opinion march layer illness can nest fence top debate monitor supreme noble'

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = True
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
PROOF_FILE_NAME = 'proof_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in tqdm(range(NUMBER_OF_PARTICIPANTS)):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    claim_item['message'] = f"{claim_item['bostrom_address']}:QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV"

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_without_proof_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

  0%|          | 0/1000 [00:00<?, ?it/s]

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1000000,john16491805420,QmYkvRCFomDU2kTS8tPHyc2dq8zbH5sGX3UkSdNwhcA3KU,0x53aab6fbd57f81474d5f982b7283db51de3c8317,0x5d9bb0a41022bc64517176c2b7683eb13db0cc3099ff...,bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5,cosmos1k7expqvjs3d9d02vveca4ugx2dhenprunnmkjn,exit cash extra kiss night bacon input tackle ...,bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5...,0x659812efcc33bd31bb6dd313305b701a9e964ffe4819...,{'pub_key': 'A1gm6WiIDTYeIyt9QSSGcauYKb35OCCfy...,eyJwdWJfa2V5IjoiQTFnbTZXaUlEVFllSXl0OVFTU0djYX...
1,1000000,john16491805421,QmcgbLatva5ByF9E9PEkvWD8MGFLkn4ysPWEYzmhnBusqN,0xde48fb7fcd78f33ebdef4956fb79e02bc70c97d6,0x103392bdb81d67f69722608de6a33d01c4bcac72f4f8...,bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f,cosmos19t84mv54avulkj5uhelnklnaq3j256j80nw2tw,brown vacuum craft mask puzzle option subway p...,bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f...,0x02ba7cd56909d60fa871286bddea9b803d737dff9e70...,{'pub_key': 'AtX2v1wR/Je2l4sRLHbsA3WbK5s0dtKDW...,eyJwdWJfa2V5IjoiQXRYMnYxd1IvSmUybDRzUkxIYnNBM1...
2,1000000,john16491805422,Qmes2KpFKKyVhMExUx9WYm1XhaXpXsJzT8nngGEZo8THNX,0x00ef128fe6552bbdbaa9ccd799348cf0bfc09004,0xbc057c174aa9f5a7ce6cd8970c5b85381e6bf05553fb...,bostrom1j7zq5gs74hlx0v0wlkfueuz2agdnm0jd95hlu3,cosmos1j7zq5gs74hlx0v0wlkfueuz2agdnm0jdx8rvzk,pool loop myth tonight submit library refuse e...,bostrom1j7zq5gs74hlx0v0wlkfueuz2agdnm0jd95hlu3...,0x28877399a194b0254e59cc51afb115d9607a6e13a266...,{'pub_key': 'AxPpSNqt0t1PuN+hsE8YGusfpjbKfShFu...,eyJwdWJfa2V5IjoiQXhQcFNOcXQwdDFQdU4raHNFOFlHdX...
3,1000000,john16491805423,QmTbBu9yNkpyvsFaRTFG2LajAtL6aeR8TxKwmvZaGtonpu,0xbbe13e3406d7eed71c67cffb687459f3b217bf9f,0xc53fbb9e5a0383a7d11f34248ab507c6749e5433a533...,bostrom183y9le3eqqy058fjkk2l23f54gxv2vkycg85qh,cosmos183y9le3eqqy058fjkk2l23f54gxv2vkymmn87s,initial police glance human okay pave tornado ...,bostrom183y9le3eqqy058fjkk2l23f54gxv2vkycg85qh...,0x78c51751be88294f022551bb8c46fb5bdda8cabd6cac...,{'pub_key': 'AuRQu3Pd0lH0wkQZBQtKanE02zIZZT1yA...,eyJwdWJfa2V5IjoiQXVSUXUzUGQwbEgwd2tRWkJRdEthbk...
4,1000000,john16491805424,QmNd8GkXwqs2avFgB2vx6anU6PqCPwMBrVaLr5h8py5gdv,0xc176ebd88f87b19f73d6bbcc638e9ca3e49c418d,0x8f34e2dac8046e1fb55d130a522c7eba2ec884271241...,bostrom1c22rcy7lq55qrcllvyxy0d4dm8we7h3ulkxe6v,cosmos1c22rcy7lq55qrcllvyxy0d4dm8we7h3uu9j2yt,excess choose sweet fluid borrow crunch unfair...,bostrom1c22rcy7lq55qrcllvyxy0d4dm8we7h3ulkxe6v...,0xcb1a4b6eed38a4d1cdd2ca93f0d621d98db04de82dbc...,{'pub_key': 'AlqmKMZ7WV49o9daIYFl286RDKRCxQ7Cf...,eyJwdWJfa2V5IjoiQWxxbUtNWjdXVjQ5bzlkYUlZRmwyOD...
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1000000,john1649180542995,QmPPN4uXU5pnXHgv6QeJANmUUQ91pzWSdvwaA9E42mGntU,0x1e1824ada074f0afdca15a6c57f30b5f028707bb,0xae23a16ea4e551a50213ff14106551e025772c663236...,bostrom14x6c0nt3l7e4xxvufapmdpch8gq6ysepwc79qu,cosmos14x6c0nt3l7e4xxvufapmdpch8gq6ysepdt2k7m,select clap panic beauty leave youth glory riv...,bostrom14x6c0nt3l7e4xxvufapmdpch8gq6ysepwc79qu...,0x1993f68d904904bd4630966e6d18a7ecbdbcb03cd7bc...,{'pub_key': 'AiMAqhMz1czTJN634ySX8Z2jHRYFytTMC...,eyJwdWJfa2V5IjoiQWlNQXFoTXoxY3pUSk42MzR5U1g4Wj...
996,1000000,john1649180542996,QmdzN3Cw1tRoi2DrCp27VGCDsJX6awPK8JKoMYQTkLCtGN,0xcf800ad16bf6144d60651b87dca581c78572ef1f,0xf7cd685f09310132680449af21d83770628b2c05c985...,bostrom18605553ld5nvw5yvf37n20asz5uy62xaxqpyxw,cosmos18605553ld5nvw5yvf37n20asz5uy62xa9n4hcf,push extend egg lobster country rug drastic im...,bostrom18605553ld5nvw5yvf37n20asz5uy62xaxqpyxw...,0x272bb707e76d72427aeb1ab01cd68f46da004f98c6c3...,{'pub_key': 'A5Ng/mcdI0YN5X4iqsA8LHugEHP1kz1o5...,eyJwdWJfa2V5IjoiQTVOZy9tY2RJMFlONVg0aXFzQThMSH...
997,1000000,john1649180542997,QmXrzBrT8hfqXVRST6ELiRRaBgibpbUzXgGaLLxQQog7sF,0x042b7342c7438babe46ac89a3d2329f07fad4df6,0x09022387bd76729fe16396ad409144e36ef25f537849...,bostrom1ywr569ke

### Create Merkle Tree, Get Root and Proofs

In [3]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]
root_source_list.append({'address': '0xF2749114FeaAD68854E01C8eE762C7170532FdfD'.lower(), 'amount': '10000000'})
with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [4]:
root_and_proofs_output, root_and_proofs_error = execute_bash(
    f'export NODE_OPTIONS=--max_old_space_size=4096 && '
    f'yarn start --input {ROOT_SOURCE_FILE_NAME} --output {PROOF_FILE_NAME}')
print(root_and_proofs_output)
with open(PROOF_FILE_NAME, 'r') as proof_file:
    root_and_proof_json = json.load(proof_file)
root = root_and_proof_json['merkle_root']
proofs_df = pd.DataFrame(root_and_proof_json['proofs'])
print(f'Merkle root: {root}')
print(f'Number of addresses in the Merkle tree: {len(proofs_df)}')

yarn run v1.22.17
$ ts-node index.ts --input root_testing_source_1000_addresses.json --output proof_testing_1000_addresses.json
Merkle root: 4beb38300c23f5b1834b36a5a79f0e2a61827388077d4d44e9bb88bc6cb47638
Number of addresses in the Merkle tree: 1999
Done in 4.27s.

Merkle root: 4beb38300c23f5b1834b36a5a79f0e2a61827388077d4d44e9bb88bc6cb47638
Number of addresses in the Merkle tree: 1999


In [5]:
NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

tasks = set(
    (
        (ROOT_SOURCE_FILE_NAME,
         f'temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(NUMBER_OF_PARTICIPANTS * 2 + 1, (i + 1) * BASH_SIZE + 1))
        for i in range(ceil(NUMBER_OF_PARTICIPANTS/BASH_SIZE))
    )
)
print(tasks)
with Pool(processes=NUMBER_OF_THREADS) as pool:
    res = pool.starmap(get_proofs, tasks)
assert res == [True] * len(res)

{('root_testing_source_1000_addresses.json', 'temp/proofs_0.json', 0, 1001)}
yarn run v1.22.17
$ ts-node index.ts --input root_testing_source_1000_addresses.json --output temp/proofs_0.json --start_index 0 --end_index 1001
Merkle root: 4beb38300c23f5b1834b36a5a79f0e2a61827388077d4d44e9bb88bc6cb47638
Number of addresses in the Merkle tree: 1001
Done in 3.03s.



In [6]:
roots = []
proofs_df = pd.DataFrame(columns=['address', 'amount', 'proof'])
for task in tasks:
    with open(task[1], 'r') as proof_file:
        root_and_proof_json = json.load(proof_file)
    roots.append(root_and_proof_json['merkle_root'])
    proofs_df = proofs_df.append(pd.DataFrame(root_and_proof_json['proofs']), ignore_index=True)
assert roots == [roots[0]] * len(roots)
proofs_df

,address,amount,proof
0,0x53aab6fbd57f81474d5f982b7283db51de3c8317,1000000,[161e04d67252fe71120c2e00d55501f8c22a95564d5d8...
1,0xde48fb7fcd78f33ebdef4956fb79e02bc70c97d6,1000000,[6a387858cf1b24bb77c557395634dd684547064aa4984...
2,0x00ef128fe6552bbdbaa9ccd799348cf0bfc09004,1000000,[cad95c432671e2767c87836f2971bb3634e055153e69b...
3,0xbbe13e3406d7eed71c67cffb687459f3b217bf9f,1000000,[961a5da54f2c4ec9b3e2e29297f2d8cd6c7161503cb9d...
4,0xc176ebd88f87b19f73d6bbcc638e9ca3e49c418d,1000000,[d4cad2b1e32ae59d7c96d7be79bd8acfd27804c87661d...
...,...,...,...
996,0xcf800ad16bf6144d60651b87dca581c78572ef1f,1000000,[a1959cdf7524769b9385e53ce9efdbe1989a15847483d...
997,0x042b7342c7438babe46ac89a3d2329f07fad4df6,1000000,[a3eceb3255a10e3aa743b716545651e1b08678264fa29...
998,0x967e13eb434842b6040666b44ee6aa47db2d35d0,1000000,[2a5b28037d80215e4574abf38c871afbf1c4c607221f1...
999,0xa92f2a2d27a1f6329c15bdada24c86ac2f3dd6c9,1000000,[51390b99be7fc69c938e543dbcf2d0a3538da10624ee7...


In [7]:
cosmos_proofs_df = proofs_df[proofs_df.address.str.startswith('cosmos')][['address', 'proof']]
ethereum_proofs_df = proofs_df[proofs_df.address.str.startswith('0x')][['address', 'proof']]

claims_with_proofs_df = claims_df\
                .merge(
                    ethereum_proofs_df.rename(columns={'address': 'ethereum_address', 'proof': 'ethereum_proof'}),
                    how='left',
                    on='ethereum_address')\
                .merge(
                    cosmos_proofs_df.rename(columns={'address': 'cosmos_address', 'proof': 'cosmos_proof'}),
                    how='left',
                    on='cosmos_address')

claims_with_proofs_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_with_proofs_df.head()

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature,ethereum_proof,cosmos_proof
0,1000000,john16491805420,QmYkvRCFomDU2kTS8tPHyc2dq8zbH5sGX3UkSdNwhcA3KU,0x53aab6fbd57f81474d5f982b7283db51de3c8317,0x5d9bb0a41022bc64517176c2b7683eb13db0cc3099ff...,bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5,cosmos1k7expqvjs3d9d02vveca4ugx2dhenprunnmkjn,exit cash extra kiss night bacon input tackle ...,bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5...,0x659812efcc33bd31bb6dd313305b701a9e964ffe4819...,{'pub_key': 'A1gm6WiIDTYeIyt9QSSGcauYKb35OCCfy...,eyJwdWJfa2V5IjoiQTFnbTZXaUlEVFllSXl0OVFTU0djYX...,[161e04d67252fe71120c2e00d55501f8c22a95564d5d8...,[d27462aa80375ce17211fc423e757fafa63a68d4c3cfc...
1,1000000,john16491805421,QmcgbLatva5ByF9E9PEkvWD8MGFLkn4ysPWEYzmhnBusqN,0xde48fb7fcd78f33ebdef4956fb79e02bc70c97d6,0x103392bdb81d67f69722608de6a33d01c4bcac72f4f8...,bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f,cosmos19t84mv54avulkj5uhelnklnaq3j256j80nw2tw,brown vacuum craft mask puzzle option subway p...,bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f...,0x02ba7cd56909d60fa871286bddea9b803d737dff9e70...,{'pub_key': 'AtX2v1wR/Je2l4sRLHbsA3WbK5s0dtKDW...,eyJwdWJfa2V5IjoiQXRYMnYxd1IvSmUybDRzUkxIYnNBM1...,[6a387858cf1b24bb77c557395634dd684547064aa4984...,NaN
2,1000000,john16491805422,Qmes2KpFKKyVhMExUx9WYm1XhaXpXsJzT8nngGEZo8THNX,0x00ef128fe6552bbdbaa9ccd799348cf0bfc09004,0xbc057c174aa9f5a7ce6cd8970c5b85381e6bf05553fb...,bostrom1j7zq5gs74hlx0v0wlkfueuz2agdnm0jd95hlu3,cosmos1j7zq5gs74hlx0v0wlkfueuz2agdnm0jdx8rvzk,pool loop myth tonight submit library refuse e...,bostrom1j7zq5gs74hlx0v0wlkfueuz2agdnm0jd95hlu3...,0x28877399a194b0254e59cc51afb115d9607a6e13a266...,{'pub_key': 'AxPpSNqt0t1PuN+hsE8YGusfpjbKfShFu...,eyJwdWJfa2V5IjoiQXhQcFNOcXQwdDFQdU4raHNFOFlHdX...,[cad95c432671e2767c87836f2971bb3634e055153e69b...,NaN
3,1000000,john16491805423,QmTbBu9yNkpyvsFaRTFG2LajAtL6aeR8TxKwmvZaGtonpu,0xbbe13e3406d7eed71c67cffb687459f3b217bf9f,0xc53fbb9e5a0383a7d11f34248ab507c6749e5433a533...,bostrom183y9le3eqqy058fjkk2l23f54gxv2vkycg85qh,cosmos183y9le3eqqy058fjkk2l23f54gxv2vkymmn87s,initial police glance human okay pave tornado ...,bostrom183y9le3eqqy058fjkk2l23f54gxv2vkycg85qh...,0x78c51751be88294f022551bb8c46fb5bdda8cabd6cac...,{'pub_key': 'AuRQu3Pd0lH0wkQZBQtKanE02zIZZT1yA...,eyJwdWJfa2V5IjoiQXVSUXUzUGQwbEgwd2tRWkJRdEthbk...,[961a5da54f2c4ec9b3e2e29297f2d8cd6c7161503cb9d...,NaN
4,1000000,john16491805424,QmNd8GkXwqs2avFgB2vx6anU6PqCPwMBrVaLr5h8py5gdv,0xc176ebd88f87b19f73d6bbcc638e9ca3e49c418d,0x8f34e2dac8046e1fb55d130a522c7eba2ec884271241...,bostrom1c22rcy7lq55qrcllvyxy0d4dm8we7h3ulkxe6v,cosmos1c22rcy7lq55qrcllvyxy0d4dm8we7h3uu9j2yt,excess choose sweet fluid borrow crunch unfair...,bostrom1c22rcy7lq55qrcllvyxy0d4dm8we7h3ulkxe6v...,0xcb1a4b6eed38a4d1cdd2ca93f0d621d98db04de82dbc...,{'pub_key': 'AlqmKMZ7WV49o9daIYFl286RDKRCxQ7Cf...,eyJwdWJfa2V5IjoiQWxxbUtNWjdXVjQ5bzlkYUlZRmwyOD...,[d4cad2b1e32ae59d7c96d7be79bd8acfd27804c87661d...,NaN


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [8]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

Name subgraph contract address: bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6
Avatar subgraph contract address: bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq
Proof subgraph contract address: bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4


### Instantiate Passport Contract

In [9]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra


### Set executor in the Subgraph Contracts

In [10]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [11]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom16mmtey75hqg7z48hyddvrrqs6xpaa78yq82zczmk4y7w60fj60ds56q6n8


### Initiate Class of Output Parsing

In [12]:
parse_output = ParseOutput(ipfs_client=ipfs_client,
                           address_dict= {
                               gift_contract_address: 'Gift Contract',
                               passport_contract_address: 'Passport Contract',
                               WALLET_ADDRESS: 'Passport Owner Address',
                               name_subgraph_contract_address: 'Name Subgraph Contract',
                               avatar_subgraph_contract_address: 'Avatar Subgraph Contract',
                               proof_subgraph_contract_address: 'Proof Subgraph Contract'})

### Register Merkle Root

In [13]:
root_register_output = execute_contract(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        from_address=WALLET_ADDRESS,
                                        contract_address=gift_contract_address)
parse_output.parse_contract_execution_json(root_register_output)


Events

execute
	execute contract: Gift Contract

message from Passport Owner Address wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Gift Contract
	action: register_merkle_root
	merkle_root: 4beb38300c23f5b1834b36a5a79f0e2a61827388077d4d44e9bb88bc6cb47638
Gas used: 117,294
Tx hash: E9BC5972D6FE9142342E86B36B8A8034E17B21667446DEED24BC18EFC4E38299


#### Get Merkle Root form the Gift Contract

In [14]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))

Gift contract bostrom16mmtey75hqg7z48hyddvrrqs6xpaa78yq82zczmk4y7w60fj60ds56q6n8
{'data': {'merkle_root': '4beb38300c23f5b1834b36a5a79f0e2a61827388077d4d44e9bb88bc6cb47638'}}


### Send coins to new addresses

In [15]:
bostrom_addresses = claims_with_proofs_df.bostrom_address.to_list()
NUMBER_ADDRESSES_IN_SENDING_CHUNK = 1000
bostrom_addresses_chunks = [bostrom_addresses[i: i+ NUMBER_ADDRESSES_IN_SENDING_CHUNK] for i in range(0, len(bostrom_addresses), NUMBER_ADDRESSES_IN_SENDING_CHUNK)]

for bostrom_addresses_item in bostrom_addresses_chunks:
    send_output = send_coins(
            from_seed=WALLET_SEED,
            to_addresses=claims_with_proofs_df.bostrom_address.to_list(),
            amounts=[1] * len(claims_with_proofs_df.bostrom_address.to_list()),
            gas=min(71000 * len(claims_with_proofs_df.bostrom_address.to_list()), int(23e6)),
            display_data=DISPLAY_TX_EXECUTION)
    parse_output.parse_contract_execution_json(contract_execution_json=send_output)


Events

coin received
	receiver: bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5
	amount: 1boot
	receiver: bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f
	amount: 1boot
	receiver: bostrom1j7zq5gs74hlx0v0wlkfueuz2agdnm0jd95hlu3
	amount: 1boot
	receiver: bostrom183y9le3eqqy058fjkk2l23f54gxv2vkycg85qh
	amount: 1boot
	receiver: bostrom1c22rcy7lq55qrcllvyxy0d4dm8we7h3ulkxe6v
	amount: 1boot
	receiver: bostrom1wumyzej3ncvzqd90vda0f3d3jztqe9mu32krz3
	amount: 1boot
	receiver: bostrom1xc2uekvacet25macjd4pqayy6g9pvfd8ww62fq
	amount: 1boot
	receiver: bostrom1tm9zvye855mjnadwhg2flktgvlqqr7n93gsr8z
	amount: 1boot
	receiver: bostrom1mdcrs5h696qs8e258wthgmjtc7exezg48tez9e
	amount: 1boot
	receiver: bostrom1z9xd2hvvkce23anurs0vnkxxf98ukwh7wprzl2
	amount: 1boot
	receiver: bostrom1zlsmu65zf3hfzhwp2nkugf39f8jrpa28l2cga9
	amount: 1boot
	receiver: bostrom1x9gypmwn7r53rkkrnp9szwn56t3um8ydjdpx7q
	amount: 1boot
	receiver: bostrom1rq03g02qswr9lqzwhv2jnaz9vg88a86pt3y02f
	amount: 1boot
	receiver: bostrom1yqe4x3zns

### Create Passports

In [16]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"create_passport":{"avatar":claim_row["avatar"], "nickname":claim_row["nickname"]}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=500000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        create_passport_json = create_passport(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(create_passport_json, row=row)
    else:
        create_passport(row)

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Nickname
	Nickname -> Passport Owner Address
	neuron: Name Subgraph Contract

	Passport Owner Address -> Avatar
	Avatar -> Passport Owner Address
	neuron: Avatar Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Name Subgraph Contract
	execute contract: Avatar Subgraph Contract

message from bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5 wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 209
Gas used: 407,208
Tx hash: 27F13AB3DB73EE25DB27004DF1ECA791905623AD426FC5571F83EB03AFD5B054


### Proof (Add) Address to Passports

In [17]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"proof_address": {"address":claim_row[network + "_address"], "nickname": claim_row["nickname"], "signature": claim_row[network + "_message_signature"]}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        proof_ethereum_address_json = proof_address(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(proof_ethereum_address_json, row=row)

        proof_cosmos_address_json = proof_address(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(proof_cosmos_address_json, row=row)
    else:
        proof_address(row)
        proof_address(row, network='cosmos')

0it [00:00, ?it/s]


Events

cyberlinks
	Passport Owner Address -> Ethereum Address
	Ethereum Address -> Passport Owner Address
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16491805421
	address: 0xde48fb7fcd78f33ebdef4956fb79e02bc70c97d6
Gas used: 265,000
Tx hash: 4AB4ADD58241DD2B3007503FD7F8A457AB88CE06055FE268CD73BF3E70FFA0FE

Events

cyberlinks
	Passport Owner Address -> Cosmos Address
	Cosmos Address -> Passport Owner Address
	neuron: Proof Subgraph Contract


execute
	execute contract: Passport Contract
	execute contract: Proof Subgraph Contract

message from bostrom19t84mv54avulkj5uhelnklnaq3j256j8vq6e4f wasm /cosmwasm.wasm.v1.MsgExecuteContract

reply
	reply contract: Passport Contract

wasm
	_contr

### Claim

In [18]:
def claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"claim": {"nickname": claim_row['nickname'], "gift_claiming_address": claim_row[network + "_address"], "gift_amount": str(claim_row['amount']), "proof": claim_row[network + "_proof"]}},
        contract_address=gift_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        claim_ethereum_json = claim(row, display_data=True)
        parse_output.parse_contract_execution_json(claim_ethereum_json)

        claim_cosmos_json = claim(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(claim_cosmos_json)
    else:
        claim_ethereum_json_2 = claim(row)
        parse_output.parse_contract_execution_json(claim_ethereum_json_2)

        claim_cosmos_json_2 = claim(row, network='cosmos')
        parse_output.parse_contract_execution_json(claim_cosmos_json_2)

0it [00:00, ?it/s]

MsgExecuteContract(sender='bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5', contract='bostrom16mmtey75hqg7z48hyddvrrqs6xpaa78yq82zczmk4y7w60fj60ds56q6n8', execute_msg={'claim': {'nickname': 'john16491805420', 'gift_claiming_address': '0x53aab6fbd57f81474d5f982b7283db51de3c8317', 'gift_amount': '1000000', 'proof': ['161e04d67252fe71120c2e00d55501f8c22a95564d5d8ebc6bf48b10caad8d07', '7d8103c81cfc5d4787dca5edcc490b5ca19829f2daaba858fc983db84cbe6257', 'a8b9da4bf8622363d383156a88f4bcca3f819b0b339236cd49a9962e94be623c', '430baa2e4dae09ead61ba3402bbd621bf3acf4c0feb58e1ac88dcdaa3e99d08f', '11412ddba6be38b853cd84edceebacbe02165e8b597f7fd5f16c21c3c923b6ed', 'a321051347d21c788bbd54c192521857733fbcbdda8b645107478bca464099cd', '622dbee85410571169ead48be8946571f462b26c34405540d0899d874dea15fb', '36e3fa2c87986f4db50710458ce13451846a4caf9b4f07d6bfd7b3954f0788d8', 'b568a56dbdd8bdf3813edfa0b447ed4b0fa6d4febde8778119a652bde0c58d29', '5cdc291d151b402713a47d46c6467a39be9048c2337419006554f5bb73320cd2', '900

### Release Gift

In [19]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_contract_sdk(
        execute_msg={"release": {"gift_address": claim_row[network + "_address"]}},
        contract_address=gift_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=400000,
        display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 0:
        release_ethereum_json = release(row, display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(release_ethereum_json)

        release_cosmos_json = release(row, network='cosmos', display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(release_cosmos_json)
    else:
        release(row)
        release(row, network='cosmos')

0it [00:00, ?it/s]


Events

coin received
	receiver: bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5 wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5
	sender: Gift Contract
	amount: 1290000boot

wasm
	_contract_address: Gift Contract
	action: release
	address: bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5
	gift_address: 0x53aab6fbd57f81474d5f982b7283db51de3c8317
	stage: 9
	amount: 1290000
Gas used: 148,799
Tx hash: E1C3B0B2FFFB35A11A144B2A03655F20BBBF52743B978CAEF0D8B9399C25F462

Events

coin received
	receiver: bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5
	amount: 1290000boot

coin spent
	spender: Gift Contract
	amount: 1290000boot

execute
	execute contract: Gift Contract

message from bostrom1k7expqvjs3d9d02vveca4ugx2dhenprusq09v5 wasm /cosmwasm.wasm.v1.MsgExec

## Passport NFT testing
### Transfer Passport

In [20]:
def get_passport_id(bostrom_address: str) -> str:
    return query_contract(query=f'''{{"portid": {{"portid": "{bostrom_address}"}}}}''',
                          contract_address=passport_contract_address)['data']['portid']

218

In [25]:
def transfer_passport(claim_row: pd.Series, token_id: str,  to_address: str = '', display_data: bool = False):
    if to_address == '':
        to_address = claim_row['bostrom_address']
    return execute_contract_sdk(
        execute_msg={"transfer_nft": {"recipient": to_address, "token_id": str(token_id)}},
        contract_address=passport_contract_address,
        mnemonic=claim_row['cosmos_seed'],
        gas=500000,
        display_data=display_data)


for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        transfer_passport_json = transfer_passport(
            row, 
            token_id=get_passport_id(row['bostrom_address']), 
            to_address=WALLET_ADDRESS, 
            display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(
            transfer_passport_json, 
            row=row)
        break
    else:
        transfer_passport_json = transfer_passport(
            row, 
            token_id=get_passport_id(WALLET_ADDRESS), 
            to_address=WALLET_ADDRESS)

0it [00:00, ?it/s]


Events
failed to execute message; message index: 0: Unauthorized: execute wasm contract failed
Gas used: 136,631
Tx hash: 62BCE1E432161B2351BFE4F76B001A2C961E10C22FC99F6FBD45FC92D3CD74B9


In [24]:
parse_output.parse_contract_execution_json(transfer_passport_json, row=row)

NameError: name 'transfer_passport_json' is not defined

### Burn

In [23]:
def burn_passport(claim_row: pd.Series, token_id: str,  display_data: bool = False):
    return execute_contract(execute_query=f'''{{"burn":{{"token_id":"{token_id}"}}}}''',
                            contract_address=passport_contract_address,
                            gas=400000,
                            display_data=display_data)

for index, row in tqdm(claims_with_proofs_df[:NUMBER_OF_ACTIVATED_PARTICIPANTS].iterrows()):
    if index == 1:
        burn_passport_json = burn_passport(row, token_id=str(int(get_passport_id(WALLET_ADDRESS)) - 1), display_data=DISPLAY_TX_EXECUTION)
        parse_output.parse_contract_execution_json(burn_passport_json, row=row)
        break
    else:
        create_passport_json = burn_passport(row, token_id=get_passport_id(WALLET_ADDRESS))

0it [00:00, ?it/s]


Events
failed to execute message; message index: 0: Unauthorized: execute wasm contract failed
Gas used: 130,402
Tx hash: 474F39DA3CF33F7328144D8F0C744DC54ED4E17E4500EF75C21D87CF64603A7D


### Add tests for Passport contract
#### create_passport +
#### update_name
#### update_avatar
#### proof_address +
#### remove_address
#### set_minter
#### set_owner
#### set_active
#### set_subgraphs
#### transfer_nft +
#### send_nft
#### mint
#### burn +
#### approve
#### approve_all
#### revoke
#### revoke_all
#### Expirations
##### at_height
##### at_time
##### never

### Add tests for Gift contract
#### update_owner
#### update_passport_addr
#### update_target
#### register_merkle_root +
#### claim +
#### release +

### Add tests for subgraph contract
#### update_owner
#### update_executer +
